# First attempt at using CANINE

This notebook is a first test to use CANINE, the idea is to reproduce the results presented in the paper, as well as doing other experiments.

In [1]:
import os

from datasets import load_dataset
from transformers import (
    CanineForSequenceClassification,
    CanineTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
)

In [2]:
tokenizer = CanineTokenizer.from_pretrained("google/canine-c")
model = CanineForSequenceClassification.from_pretrained("google/canine-c", num_labels=3)

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Some weights of CanineForSequenceClassification were not initialized from the model checkpoint at google/canine-c and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
fr_dataset = load_dataset("xnli", "fr")

Found cached dataset xnli (/home/gwatk/.cache/huggingface/datasets/xnli/fr/1.1.0/818164464f9c9fd15776ca8a00423b074344c3e929d00a2c1a84aa5a50c928bd)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
training_args = TrainingArguments(
    output_dir="../results",  # output directory
    num_train_epochs=3,  # total number of training epochs
    learning_rate=2e-05,
    per_device_train_batch_size=4,  # batch size per device during training
    gradient_accumulation_steps=8,  # to double the effective batch size for
    warmup_ratio=0.06,  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,  # strength of weight decay
    fp16=False,
)

In [5]:
def tokenize_dataset(sample):
    return tokenizer(text=sample["premise"], text_pair=sample["hypothesis"])

In [6]:
dataset = fr_dataset.map(tokenize_dataset, batched=False, with_indices=False)

Loading cached processed dataset at /home/gwatk/.cache/huggingface/datasets/xnli/fr/1.1.0/818164464f9c9fd15776ca8a00423b074344c3e929d00a2c1a84aa5a50c928bd/cache-5e8e03d9297660b4.arrow
Loading cached processed dataset at /home/gwatk/.cache/huggingface/datasets/xnli/fr/1.1.0/818164464f9c9fd15776ca8a00423b074344c3e929d00a2c1a84aa5a50c928bd/cache-4cb1b61b2c3743f7.arrow
Loading cached processed dataset at /home/gwatk/.cache/huggingface/datasets/xnli/fr/1.1.0/818164464f9c9fd15776ca8a00423b074344c3e929d00a2c1a84aa5a50c928bd/cache-05a20d4fd5b7b824.arrow


In [7]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [9]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

In [10]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `CanineForSequenceClassification.forward` and have been ignored: hypothesis, premise. If hypothesis, premise are not expected by `CanineForSequenceClassification.forward`,  you can safely ignore this message.
/home/gwatk/.cache/pypoetry/virtualenvs/mva-snlp-canine-JqMbT94n-py3.10/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 392702
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 8
  Total optimization steps = 36816
  Number of trainable parameters = 132085251


  0%|          | 0/36816 [00:00<?, ?it/s]

KeyboardInterrupt: 